In [ ]:
import xarray as xr
import holoviews as hv
import numpy as np
hv.extension('bokeh')

Load the iso-thermally binned data

In [ ]:
W = xr.open_dataset("../data/processed/binned.nc")['uY']
W

x = W.stack(features=['z','temp']).transpose('t', 'features')

# EOF Analysis

In [ ]:
from sklearn.decomposition import PCA
import pandas as pd

pca = PCA(n_components=10).fit(x)

This is the explained variance ratios for each mode.

In [ ]:
hv.Bars(pca.explained_variance_ratio_)

In [ ]:
coords =  (pd.Index(range(10), name='m'), x.indexes['features'])
eofs = xr.DataArray(pca.components_, coords).unstack('dim_1')

This is what the eofs look like

In [ ]:
%%opts Image[colorbar=True](cmap='RdBu_r')
hv_ds = hv.Dataset(eofs.to_dataset(name='eof'), kdims=['m','z','temp'])
hv_ds.to(hv.Image, ["temp", "z"])

What do the principal component time series look like.

In [ ]:
y = pca.transform(x)

In [ ]:
%%opts Curve[width=600]
hv.HoloMap({k: hv.Curve(y_k[:1000]) for k, y_k in enumerate(y.T)})

You can see that there is not good time-scale separation, but that the first mode basically looks just like the Nusselt nnumber time series.

# DMD

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.pipeline import make_pipeline

x0, x1 = x[:-1,:].data, x[1:,:].data

pcr = make_pipeline(PCA(n_components=10), LinearRegression())
pcr.fit(x0, x1)

Now what about the linear model?

In [ ]:
I = np.eye(x0.shape[1])

A = pcr.predict(I) - pcr.predict(I*0)

Let's look at the transformation matrix of this linear operation

In [ ]:
%%opts Image
hv.Image(np.abs(A)>.01)

What is the R2 of this fit?

In [ ]:
pcr.score(x0, x1)

This is probably worse than the persistence forecast

In [ ]:
def r2_score_peristence(ytrue, ypred, ynull):
    def ss(x):
        return np.sum(x**2)
    return 1 - ss(ytrue-ypred)/ss(ynull-ytrue)



r2_score_peristence(x1, pcr.predict(x0), x0)

Actually it does a lot better than persistence forecast.

Now, let's look at the eignvalues of $A$. 

In [ ]:
%%opts Scatter(size=4)
lam = np.linalg.eigvals(A)
hv.Scatter((lam.real, lam.imag), kdims=['Re'], vdims=['Im']) * hv.Ellipse(0,0, 2.0)

They all lie within the unit circle in the complex plane, which is very good. The linear problem can be solved by taking the log of these eigenvalues.

In [ ]:
lam_pos_mask = np.abs(lam) > 1e-10
sum(lam_pos_mask)

There are only 10 eigenvalues above 0. This is exactly the number of PCA modes we selected.

Let's plot the frequencies:

In [ ]:
lam = np.ma.array(lam, mask=~lam_pos_mask)

In [ ]:
%%opts Scatter(size=4) VLine(line_color="k")

loglam = np.log(lam[lam_pos_mask])
hv.VLine(0.0) * hv.Scatter((loglam.real, loglam.imag), kdims=['Re'], vdims=['Im']) 

There is one large growing mode.

# Ridge regression

In [ ]:
from sklearn.preprocessing import StandardScaler

ridge = make_pipeline(StandardScaler(), Ridge(10.0))
ridge.fit(x0, x1)

In [ ]:
A_ridge = ridge.predict(I) - ridge.predict(0*I)

In [ ]:
ridge.score(x0, x1)

In [ ]:
%%opts Scatter(size=1)
lam = np.linalg.eigvals(A_ridge)
hv.Scatter((lam.real, lam.imag), kdims=['Re'], vdims=['Im']) * hv.Ellipse(0,0, 2.0)

There is not good time-scale separation here because the imaginary components of the eigenvalues are uniformly distributed accross the imaginary axis. It appears the SVD approach gives a more robust estimation of the eigenvalues than the ridge regression does.

In [ ]:
# freq, bins = np.histogram(lam.imag, 10)
hv.Scatter(lam.imag)